# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from citipy import citipy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
csv_path = "Output/2020Oct26weather.csv"
city_weather_df=pd.read_csv(csv_path, encoding="utf-8")
city_weather_df.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind_Speed (mph)
0,atuona,-9.800000,-139.033333,78.91,76.0,86.0,19.69
1,rikitea,-23.120278,-134.969167,75.34,85.0,100.0,19.24
2,karauzyak,43.027500,60.012500,48.20,61.0,0.0,4.70
3,punta arenas,-53.150000,-70.916667,55.40,37.0,20.0,34.45
4,pathein,16.783333,94.733333,75.83,90.0,0.0,4.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
locations = city_weather_df[["Latitude", "Longitude"]]
humidity = city_weather_df["Humidity (%)"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
ideal_weather_df=city_weather_df.loc[(city_weather_df["Humidity (%)"]>20) & (city_weather_df["Humidity (%)"]<50) & (city_weather_df["Cloudiness (%)"]<20) & (city_weather_df["Wind_Speed (mph)"]<10) & (city_weather_df["Temperature (F)"]>=50) & (city_weather_df["Temperature (F)"]<=60),:].reset_index(drop=True)

Country=[]
for index, row in test_df.iterrows():
    Country.append(citipy.nearest_city(row["Latitude"],row["Longitude"]).country_code.upper())
ideal_weather_df["Country"]=Country

print(ideal_weather_df.count())
ideal_weather_df.head()

City                10
Latitude            10
Longitude           10
Temperature (F)     10
Humidity (%)        10
Cloudiness (%)      10
Wind_Speed (mph)    10
Country             10
dtype: int64


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind_Speed (mph),Country
0,torbat-e jam,35.244000,60.622500,57.06,33.0,0.0,9.04,IR
1,nuristan,34.950000,70.400000,51.60,40.0,0.0,1.79,AF
2,newport,44.636944,-124.052222,51.55,39.0,1.0,8.05,US
3,ardistan,33.376100,52.369400,53.47,38.0,0.0,2.73,IR
4,north bend,43.406667,-124.223056,56.32,37.0,1.0,4.70,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
hotel_df=ideal_weather_df
hotels=[]
latlong=[]
target_search = "hotel"
radius=5000

for i, row in hotel_df.iterrows():
    Lat=ideal_weather_df["Latitude"][i]
    Long=ideal_weather_df["Longitude"][i]

    coords=f"{Lat},{Long}"

    params = {
        "location": coords,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params).json()
    try:
        latlong.append((response['results'][0]['geometry']['location']['lat'],response['results'][0]['geometry']['location']['lng']))
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("N/A")

hotel_df["Hotel Name"]=hotels
hotel_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind_Speed (mph),Country,Hotel Name
0,torbat-e jam,35.244000,60.622500,57.06,33.0,0.0,9.04,IR,Pardisan Hotel 🏩
1,nuristan,34.950000,70.400000,51.60,40.0,0.0,1.79,AF,N/A
2,newport,44.636944,-124.052222,51.55,39.0,1.0,8.05,US,"Hallmark Resort Hotel | Newport, Oregon"
3,ardistan,33.376100,52.369400,53.47,38.0,0.0,2.73,IR,Jahangardi Restaurant
4,north bend,43.406667,-124.223056,56.32,37.0,1.0,4.70,US,The Mill Casino • Hotel & RV Park
5,alice springs,-23.697479,133.883621,55.40,35.0,0.0,4.70,AU,DoubleTree by Hilton Hotel Alice Springs
6,wana,32.298889,69.572500,53.31,37.0,0.0,2.48,PK,United Bank Limited
7,yulara,-25.245291,130.980545,59.00,31.0,0.0,6.93,AU,Ayers Rock Resort
8,sayyan,15.166389,44.328056,53.42,34.0,0.0,4.61,YE,N/A
9,astoria,46.188056,-123.830000,50.23,49.0,1.0,6.93,US,Selina Commodore Astoria


In [68]:
hotel_df=hotel_df.loc[hotel_df["Hotel Name"]!="N/A",:]
hotel_df=hotel_df.reset_index(drop=True)
hotel_df=hotel_df[["Hotel Name","City","Country"]]
hotel_df

,Hotel Name,City,Country
0,Pardisan Hotel 🏩,torbat-e jam,IR
1,"Hallmark Resort Hotel | Newport, Oregon",newport,US
2,Jahangardi Restaurant,ardistan,IR
3,The Mill Casino • Hotel & RV Park,north bend,US
4,DoubleTree by Hilton Hotel Alice Springs,alice springs,AU
5,United Bank Limited,wana,PK
6,Ayers Rock Resort,yulara,AU
7,Selina Commodore Astoria,astoria,US


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = latlong

In [70]:
print(hotel_info)

['\n<dl>\n<dt>Name</dt><dd>Pardisan Hotel 🏩</dd>\n<dt>City</dt><dd>torbat-e jam</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Hallmark Resort Hotel | Newport, Oregon</dd>\n<dt>City</dt><dd>newport</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Jahangardi Restaurant</dd>\n<dt>City</dt><dd>ardistan</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>The Mill Casino • Hotel & RV Park</dd>\n<dt>City</dt><dd>north bend</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>DoubleTree by Hilton Hotel Alice Springs</dd>\n<dt>City</dt><dd>alice springs</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>United Bank Limited</dd>\n<dt>City</dt><dd>wana</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Ayers Rock Resort</dd>\n<dt>City</dt><dd>yulara</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Selina Commodore Astoria</dd>\n<dt>City</dt><dd>astoria<

In [71]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations,info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Looks like I'm going to need to move!